In [1]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)

In [2]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
import matplotlib.pyplot as plt
import cv2

Using TensorFlow backend.


In [3]:
batch_size = 128
nb_classes = 10
nb_epoch = 12

In [6]:
# 输入图像的维度，此处是mnist图像，因此是28*28
img_rows, img_cols = 28, 28
# 卷积层中使用的卷积核的个数
nb_filters = 32
# 池化层操作的范围
pool_size = (2,2)
# 卷积核的大小
kernel_size = (3,3)
# keras中的mnist数据集已经被划分成了60,000个训练集，10,000个测试集的形式，按以下格式调用即可
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# 后端使用tensorflow时，即tf模式下，
# 会将100张RGB三通道的16*32彩色图表示为(100,16,32,3)，
# 第一个维度是样本维，表示样本的数目，
# 第二和第三个维度是高和宽，
# 最后一个维度是通道维，表示颜色通道数
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)
# plt.imshow(X_train[0].reshape(28,28), cmap='Greys')

(28, 28)
(28, 28, 1)


In [8]:
# 将X_train, X_test的数据格式转为float32
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
# 归一化
X_train /= 255
X_test /= 255
# 打印出相关信息
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

X_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [48]:
# 将类别向量(从0到nb_classes的整数向量)映射为二值类别矩阵，
# 相当于将向量用one-hot重新编码
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

In [49]:
# 建立序贯模型
model = Sequential()

# 卷积层，对二维输入进行滑动窗卷积
# 当使用该层为第一层时，应提供input_shape参数，在tf模式中，通道维位于第三个位置
# border_mode：边界模式，为"valid","same"或"full"，即图像外的边缘点是补0
# 还是补成相同像素，或者是补1
model.add(Convolution2D(nb_filters, kernel_size[0] ,kernel_size[1],
                        border_mode='valid',
                        input_shape=input_shape))
model.add(Activation('relu'))

# 卷积层，激活函数是ReLu
model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1]))
model.add(Activation('relu'))

# 池化层，选用Maxpooling，给定pool_size，dropout比例为0.25
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.25))

# Flatten层，把多维输入进行一维化，常用在卷积层到全连接层的过渡
model.add(Flatten())

# 包含128个神经元的全连接层，激活函数为ReLu，dropout比例为0.5
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))

# 包含10个神经元的输出层，激活函数为Softmax
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

# 输出模型的参数信息
model.summary()
# 配置模型的学习过程
model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

/home/qzx/env/py3.5/lib/python3.5/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(28, 28, 1..., padding="valid")`
  # Remove the CWD from sys.path while we load stuff.
/home/qzx/env/py3.5/lib/python3.5/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3))`
  


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
activation_1 (Activation)    (None, 26, 26, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 24, 24, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 12, 12, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4608)              0         
__________

In [51]:
# 训练模型
model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch,
          verbose=1, validation_data=(X_test, Y_test))

model.save('1.h5')
# 按batch计算在某些输入数据上模型的误差
# score = model.evaluate(X_test, Y_test, verbose=0)

/home/qzx/env/py3.5/lib/python3.5/site-packages/ipykernel_launcher.py:3: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  This is separate from the ipykernel package so we can avoid doing imports until


Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 103s 2ms/step - loss: 0.0389 - acc: 0.9881 - val_loss: 0.0382 - val_acc: 0.9892
Epoch 2/12
60000/60000 [==============================] - 103s 2ms/step - loss: 0.0387 - acc: 0.9884 - val_loss: 0.0304 - val_acc: 0.9906
Epoch 3/12
60000/60000 [==============================] - 102s 2ms/step - loss: 0.0387 - acc: 0.9883 - val_loss: 0.0293 - val_acc: 0.9907
Epoch 4/12
60000/60000 [==============================] - 103s 2ms/step - loss: 0.0341 - acc: 0.9901 - val_loss: 0.0355 - val_acc: 0.9912
Epoch 5/12
60000/60000 [==============================] - 106s 2ms/step - loss: 0.0340 - acc: 0.9903 - val_loss: 0.0297 - val_acc: 0.9914
Epoch 6/12
60000/60000 [==============================] - 108s 2ms/step - loss: 0.0355 - acc: 0.9898 - val_loss: 0.0323 - val_acc: 0.9916
Epoch 7/12
60000/60000 [==============================] - 101s 2ms/step - loss: 0.0348 - acc: 0.9900 - val_loss: 0.0385 - 

In [54]:
model.load_weights('1.h5')
classes = model.evaluate(X_test, Y_test, verbose=0)
print(classes)

[0.035283134788431744, 0.9915]


In [17]:
# model.load_weights('1.h5')
import cv2
img = cv2.imread('images/timg.jpeg')

img = cv2.resize(img, (28,28), interpolation=cv2.INTER_CUBIC)
print(merged.shape)
# img_class = model.predict_classes(res)
# print(img_class)

ImportError: No module named 'IplImage'

In [18]:
import IplImage
IplImage* src
src = cvLoadImage( "images/timg.jpeg", 0)
# print(src.shape)

NameError: name 'IplImage' is not defined